# Gesture Recognition
In this project, we are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Import libraries to get started.

In [1]:
import numpy as np
import datetime
import os
import cv2

import random as rn
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization
from keras.layers import Activation, Dropout, ZeroPadding3D
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.recurrent import LSTM

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
rn.seed(30)
tf.set_random_seed(30)

train_path = './Project_data/train'
val_path = './Project_data/val'
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

## Generator Class
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
class DataGenerator:
    def __init__(self, width=120, height=120, frames=30, channel=3, 
                 crop = True, normalize = False, affine = False, flip = False, edge = False  ):
        self.width = width   # X dimension of the image
        self.height = height # Y dimesnion of the image
        self.frames = frames # length/depth of the video frames
        self.channel = channel # number of channels in images 3 for color(RGB) and 1 for Gray  
        self.affine = affine # augment data with affine transform of the image
        self.flip = flip
        self.normalize =  normalize
        self.edge = edge # edge detection
        self.crop = crop

    # Helper function to generate a random affine transform on the image
    def __get_random_affine(self): # private method
        dx, dy = np.random.randint(-1.7, 1.8, 2)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        return M

    # Helper function to initialize all the batch image data and labels
    def __init_batch_data(self, batch_size): # private method
        batch_data = np.zeros((batch_size, self.frames, self.width, self.height, self.channel)) 
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        return batch_data, batch_labels

    def __load_batch_images(self, source_path, folder_list, batch_num, batch_size, t): # private method
    
        batch_data,batch_labels = self.__init_batch_data(batch_size)
        # We will also build a agumented batch data
        if self.affine:
            batch_data_aug,batch_labels_aug = self.__init_batch_data(batch_size)
        if self.flip:
            batch_data_flip,batch_labels_flip = self.__init_batch_data(batch_size)

        #create a list of image numbers you want to use for a particular video
        img_idx = [x for x in range(0, self.frames)] 

        for folder in range(batch_size): # iterate over the batch_size
            # read all the images in the folder
            imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
            # Generate a random affine to be used in image transformation for buidling agumented data set
            M = self.__get_random_affine()
            
            #  Iterate over the frames/images of a folder to read them in
            for idx, item in enumerate(img_idx): 
                image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes  
                if self.crop:
                    image = self.__crop(image)
                # If normalize is set normalize the image else use the raw image.
                if self.normalize:
                    resized = self.__normalize(self.__resize(image))
                else:
                    resized = self.__resize(image)
                # If the input is edge detected image then use the sobelx, sobely and laplacian as 3 channel of the edge detected image
                if self.edge:
                    resized = self.__edge(resized)
                
                batch_data[folder,idx] = resized
                if self.affine:
                    batch_data_aug[folder,idx] = self.__affine(resized, M)   
                if self.flip:
                    batch_data_flip[folder,idx] = self.__flip(resized)   

            batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
            
            if self.affine:
                batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
            
            if self.flip:
                if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
                elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        if self.affine:
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
        if self.flip:
            batch_data = np.append(batch_data, batch_data_flip, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_flip, axis = 0) 

        return batch_data, batch_labels
    
    def generator(self, source_path, folder_list, batch_size): # public method
        print( 'Source path = ', source_path, '; batch size =', batch_size)
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(folder_list)//batch_size # calculate the number of batches
            for batch in range(num_batches): # we iterate over the number of batches
                # you yield the batch_data and the batch_labels, remember what does yield do
                yield self.__load_batch_images(source_path, folder_list, batch, batch_size, t) 
            
            # Code for the remaining data points which are left after full batches
            if (len(folder_list) != batch_size*num_batches):
                batch_size = len(folder_list) - (batch_size*num_batches)
                yield self.__load_batch_images(source_path, folder_list, num_batches, batch_size, t)

    # Helper function to perfom affice transform on the image
    def __affine(self, image, M):
        return cv2.warpAffine(image, M, (image.shape[0], image.shape[1]))

    # Helper function to flip the image
    def __flip(self, image):
        return np.flip(image,1)
    
    # Helper function to normalise the data
    def __normalize(self, image):
        return image/127.5-1
    
    # Helper function to resize the image
    def __resize(self, image):
        return cv2.resize(image, (self.width,self.height), interpolation = cv2.INTER_AREA)
    
    # Helper function to crop the image
    def __crop(self, image):
        if image.shape[0] != image.shape[1]:
            return image[0:120, 20:140]
        else:
            return image

    # Helper function for edge detection
    def __edge(self, image):
        edge = np.zeros((image.shape[0], image.shape[1], image.shape[2]))
        edge[:,:,0] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,0],(3,3),0),cv2.CV_64F)
        edge[:,:,1] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,1],(3,3),0),cv2.CV_64F)
        edge[:,:,2] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,2],(3,3),0),cv2.CV_64F)
        return edge

## Model Class

In [5]:
class ModelGenerator(object):
    
    @classmethod
    def c3d1(cls, input_shape, nb_classes):
        """
        Build a 3D convolutional network, based loosely on C3D.
            https://arxiv.org/pdf/1412.0767.pdf
        """
        # Model.
        model = Sequential()
        model.add(Conv3D(
            8, (3,3,3), activation='relu', input_shape=input_shape
        ))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(16, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model
    
    @classmethod
    def c3d2(cls, input_shape, nb_classes):
        """
        Build a 3D convolutional network, aka C3D.
            https://arxiv.org/pdf/1412.0767.pdf
        """        
        model = Sequential()
        # 1st layer group
        model.add(Conv3D(16, 3, 3, 3, activation='relu',
                         padding='same', name='conv1',
                         subsample=(1, 1, 1),
                         input_shape=input_shape))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               border_mode='valid', name='pool1'))
        # 2nd layer group
        model.add(Conv3D(32, 3, 3, 3, activation='relu',
                         padding='same', name='conv2',
                         subsample=(1, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool2'))
        # 3rd layer group
        model.add(Conv3D(64, 3, 3, 3, activation='relu',
                         padding='same', name='conv3a',
                         subsample=(1, 1, 1)))
        model.add(Conv3D(64, 3, 3, 3, activation='relu',
                         padding='same', name='conv3b',
                         subsample=(1, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool3'))
        # 4th layer group
        model.add(Conv3D(128, 3, 3, 3, activation='relu',
                         padding='same', name='conv4a',
                         subsample=(1, 1, 1)))
        model.add(Conv3D(128, 3, 3, 3, activation='relu',
                         padding='same', name='conv4b',
                         subsample=(1, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool4'))

        # 5th layer group
        model.add(Conv3D(256, 3, 3, 3, activation='relu',
                         padding='same', name='conv5a',
                         subsample=(1, 1, 1)))
        model.add(Conv3D(256, 3, 3, 3, activation='relu',
                         padding='same', name='conv5b',
                         subsample=(1, 1, 1)))
        model.add(ZeroPadding3D(padding=(0, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool5'))
        model.add(Flatten())

        # FC layers group
        model.add(Dense(512, activation='relu', name='fc6'))
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu', name='fc7'))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model
    
    @classmethod
    def c3d3(cls, input_shape, nb_classes):
        model = Sequential()
        model.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=input_shape, padding='same'))
        model.add(Activation('relu'))
        model.add(Conv3D(16, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
        model.add(Dropout(0.25))

        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
        model.add(Dropout(0.25))

        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model
 
    @classmethod
    def c3d4(cls, input_shape, nb_classes):
        # Define model
        model = Sequential()

        model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        #Flatten Layers
        model.add(Flatten())

        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))

        #softmax layer
        model.add(Dense(5, activation='softmax'))
        return model
    
    @classmethod
    def c3d5(cls, input_shape, nb_classes):
        # Define model
        model = Sequential()

        model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        #Flatten Layers
        model.add(Flatten())

        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))

        #softmax layer
        model.add(Dense(5, activation='softmax'))
        return model   
    
    @classmethod
    def lstm(cls, input_shape, nb_classes):
        """Build a simple LSTM network. We pass the extracted features from
        our CNN to this model predomenently."""
        # Model.
        model = Sequential()
        model.add(LSTM(2048, return_sequences=False,
                       input_shape=input_shape,
                       dropout=0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

    @classmethod
    def lrcn(cls, input_shape, nb_classes):
        """Build a CNN into RNN.
        Starting version from:
            https://github.com/udacity/self-driving-car/blob/master/
                steering-models/community-models/chauffeur/models.py
        Heavily influenced by VGG-16:
            https://arxiv.org/abs/1409.1556
        Also known as an LRCN:
            https://arxiv.org/pdf/1411.4389.pdf
        """
        model = Sequential()

        model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),
            activation='relu', padding='same'), input_shape=input_shape))
        model.add(TimeDistributed(Conv2D(32, (3,3),
            kernel_initializer="he_normal", activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
        model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Flatten()))

        model.add(Dropout(0.5))
        model.add(LSTM(256, return_sequences=False, dropout=0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

    @classmethod
    def mlp(cls, input_shape, nb_classes):
        """Build a simple MLP. It uses extracted features as the input
        because of the otherwise too-high dimensionality."""
        # Model.
        model = Sequential()
        model.add(Flatten(input_shape=input_shape))
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

## Trainer Function

In [6]:
def train(batch_size, num_epochs, model, train_generator, val_generator, optimiser=None):

    curr_dt_time = datetime.datetime.now()

    num_train_sequences = len(train_doc)
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    print('# batch size =', batch_size)    
    print('# epochs =', num_epochs)

    #optimizer = Adam(lr=rate) 
    #write your optimizer
    if optimiser == None:
        optimiser = Adam() 
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)
            
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=False, 
                                 save_weights_only=False, 
                                 mode='auto', 
                                 period=1)
    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
    callbacks_list = [checkpoint, LR]

    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1

    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                callbacks=callbacks_list, validation_data=val_generator, 
                validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
    
    K.clear_session()

In [7]:
from numba import cuda
def clear_cuda():
    #cuda.select_device(0)
    #cuda.close()
    pass

## Model #1

### Model 1a : Resize to 120*120,  Raw image input, No cropping, No normalisation, No agumentation, No flipped images, No edge detection

In [8]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

batch_size = 20
num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)


# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
______________________________________________________

34/34 [==============================] - 5s 136ms/step - loss: 14.5379 - categorical_accuracy: 0.0980 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00018: saving model to model_init_2018-10-2121_17_53.160408/model-00018-14.53789-0.09804-12.41093-0.23000.h5
Epoch 19/20
34/34 [==============================] - 5s 138ms/step - loss: 12.6416 - categorical_accuracy: 0.2157 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00019: saving model to model_init_2018-10-2121_17_53.160408/model-00019-12.64164-0.21569-12.41093-0.23000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
Epoch 20/20
34/34 [==============================] - 5s 136ms/step - loss: 12.1676 - categorical_accuracy: 0.2451 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00020: saving model to model_init_2018-10-2121_17_53.160408/model-00020-12.16758-0.24510-12.41093-0.23000.h5


### Model 1b : Resize to 120*120, agumentation, No flipped images, No cropping, No normalisation, No edge detection

In [9]:
train_gen = DataGenerator(affine=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
______________________________________________________

34/34 [==============================] - 5s 159ms/step - loss: 13.5898 - categorical_accuracy: 0.1569 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00018: saving model to model_init_2018-10-2121_19_44.687027/model-00018-13.58977-0.15686-13.53920-0.16000.h5
Epoch 19/20
34/34 [==============================] - 5s 153ms/step - loss: 12.7997 - categorical_accuracy: 0.2059 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00019: saving model to model_init_2018-10-2121_19_44.687027/model-00019-12.79966-0.20588-13.53920-0.16000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
Epoch 20/20
34/34 [==============================] - 6s 179ms/step - loss: 13.3527 - categorical_accuracy: 0.1716 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00020: saving model to model_init_2018-10-2121_19_44.687027/model-00020-13.35274-0.17157-13.53920-0.16000.h5


### Model 1c : Resize to 120*120, agumentation, flipped images, No normalisation, No cropping, No edge detection

In [10]:
train_gen = DataGenerator(affine=True, flip=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
______________________________________________________

34/34 [==============================] - 6s 168ms/step - loss: 12.7997 - categorical_accuracy: 0.2059 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00018: saving model to model_init_2018-10-2121_21_58.844546/model-00018-12.79966-0.20588-12.57211-0.22000.h5
Epoch 19/20
34/34 [==============================] - 6s 185ms/step - loss: 13.4317 - categorical_accuracy: 0.1667 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00019: saving model to model_init_2018-10-2121_21_58.844546/model-00019-13.43175-0.16667-12.57211-0.22000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
Epoch 20/20
34/34 [==============================] - 6s 165ms/step - loss: 12.7997 - categorical_accuracy: 0.2059 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00020: saving model to model_init_2018-10-2121_21_58.844546/model-00020-12.79966-0.20588-12.57211-0.22000.h5


### Model 1d : Resize to 120*120,  agumentation, flipped images, normalisation, No cropping, No edge detection

In [11]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
______________________________________________________

34/34 [==============================] - 6s 181ms/step - loss: 1.1060 - categorical_accuracy: 0.4804 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch 00019: saving model to model_init_2018-10-2121_24_30.405740/model-00019-1.10598-0.48039-12.08857-0.25000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 20/20
34/34 [==============================] - 6s 182ms/step - loss: 1.0769 - categorical_accuracy: 0.5229 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch 00020: saving model to model_init_2018-10-2121_24_30.405740/model-00020-1.07690-0.52288-12.08857-0.25000.h5


### Model 1e : Resize to 120*120,  agumentation, flipped images, normalisation, cropping, No edge detection

In [12]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
______________________________________________________

34/34 [==============================] - 7s 200ms/step - loss: 1.6111 - categorical_accuracy: 0.1928 - val_loss: 11.4666 - val_categorical_accuracy: 0.2200

Epoch 00019: saving model to model_init_2018-10-2121_27_14.268054/model-00019-1.61114-0.19281-11.46664-0.22000.h5
Epoch 20/20
34/34 [==============================] - 6s 186ms/step - loss: 1.6066 - categorical_accuracy: 0.1961 - val_loss: 11.4659 - val_categorical_accuracy: 0.2200

Epoch 00020: saving model to model_init_2018-10-2121_27_14.268054/model-00020-1.60659-0.19608-11.46594-0.22000.h5

Epoch 00020: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.


### Model 1f : Resize to 120*120,  agumentation, flipped images, normalisation, cropping, edge detection

In [13]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
______________________________________________________

34/34 [==============================] - 7s 204ms/step - loss: 1.0505 - categorical_accuracy: 0.5458 - val_loss: 11.1215 - val_categorical_accuracy: 0.3100

Epoch 00019: saving model to model_init_2018-10-2121_30_03.823329/model-00019-1.05052-0.54575-11.12149-0.31000.h5
Epoch 20/20
34/34 [==============================] - 7s 211ms/step - loss: 0.9034 - categorical_accuracy: 0.6111 - val_loss: 11.1215 - val_categorical_accuracy: 0.3100

Epoch 00020: saving model to model_init_2018-10-2121_30_03.823329/model-00020-0.90338-0.61111-11.12149-0.31000.h5


## Model #2

### Model 2a : Resize to 120*120,  No agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [14]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d2(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________

34/34 [==============================] - 5s 142ms/step - loss: 1.6099 - categorical_accuracy: 0.2451 - val_loss: 1.6077 - val_categorical_accuracy: 0.2100

Epoch 00017: saving model to model_init_2018-10-2121_33_08.395073/model-00017-1.60990-0.24510-1.60772-0.21000.h5
Epoch 18/20
34/34 [==============================] - 5s 138ms/step - loss: 1.6096 - categorical_accuracy: 0.1471 - val_loss: 1.6077 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2121_33_08.395073/model-00018-1.60957-0.14706-1.60770-0.21000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 19/20
34/34 [==============================] - 5s 138ms/step - loss: 1.6048 - categorical_accuracy: 0.2647 - val_loss: 1.6077 - val_categorical_accuracy: 0.2100

Epoch 00019: saving model to model_init_2018-10-2121_33_08.395073/model-00019-1.60482-0.26471-1.60769-0.21000.h5
Epoch 20/20
34/34 [==============================] - 5s 135ms/step - loss: 1.6068 - cate

### Model 2b : Resize to 120*120, agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [15]:
train_gen = DataGenerator(affine=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d2(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________

34/34 [==============================] - 5s 148ms/step - loss: 1.6102 - categorical_accuracy: 0.2304 - val_loss: 1.6090 - val_categorical_accuracy: 0.1800

Epoch 00017: saving model to model_init_2018-10-2121_35_00.467614/model-00017-1.61025-0.23039-1.60901-0.18000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 18/20
34/34 [==============================] - 5s 158ms/step - loss: 1.6091 - categorical_accuracy: 0.2108 - val_loss: 1.6090 - val_categorical_accuracy: 0.1800

Epoch 00018: saving model to model_init_2018-10-2121_35_00.467614/model-00018-1.60907-0.21078-1.60898-0.18000.h5
Epoch 19/20
34/34 [==============================] - 5s 150ms/step - loss: 1.6101 - categorical_accuracy: 0.1471 - val_loss: 1.6090 - val_categorical_accuracy: 0.1800

Epoch 00019: saving model to model_init_2018-10-2121_35_00.467614/model-00019-1.61014-0.14706-1.60897-0.18000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 2

### Model 2c : Resize to 120*120, agumentation, flipped images, No normalisation, No cropping, No edge detection

In [16]:
train_gen = DataGenerator(affine=True, flip=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d2(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________

34/34 [==============================] - 6s 187ms/step - loss: 1.6071 - categorical_accuracy: 0.1993 - val_loss: 1.6080 - val_categorical_accuracy: 0.2100

Epoch 00017: saving model to model_init_2018-10-2121_37_18.707246/model-00017-1.60712-0.19935-1.60795-0.21000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 18/20
34/34 [==============================] - 6s 189ms/step - loss: 1.6108 - categorical_accuracy: 0.2124 - val_loss: 1.6080 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2121_37_18.707246/model-00018-1.61079-0.21242-1.60795-0.21000.h5
Epoch 19/20
34/34 [==============================] - 6s 178ms/step - loss: 1.6079 - categorical_accuracy: 0.2353 - val_loss: 1.6080 - val_categorical_accuracy: 0.2100

Epoch 00019: saving model to model_init_2018-10-2121_37_18.707246/model-00019-1.60786-0.23529-1.60796-0.21000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
Epoch 

### Model 2d : Resize to 120*120, agumentation, flipped images, normalisation, No cropping, No edge detection

In [17]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d2(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________


Epoch 00017: saving model to model_init_2018-10-2121_39_56.119221/model-00017-1.60938-0.21895-1.60718-0.21000.h5
Epoch 18/20
34/34 [==============================] - 6s 189ms/step - loss: 1.6048 - categorical_accuracy: 0.2418 - val_loss: 1.6071 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2121_39_56.119221/model-00018-1.60481-0.24183-1.60715-0.21000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 19/20
34/34 [==============================] - 6s 183ms/step - loss: 1.6114 - categorical_accuracy: 0.1863 - val_loss: 1.6072 - val_categorical_accuracy: 0.2100

Epoch 00019: saving model to model_init_2018-10-2121_39_56.119221/model-00019-1.61139-0.18627-1.60716-0.21000.h5
Epoch 20/20
34/34 [==============================] - 7s 192ms/step - loss: 1.6093 - categorical_accuracy: 0.2092 - val_loss: 1.6072 - val_categorical_accuracy: 0.2100

Epoch 00020: saving model to model_init_2018-10-2121_39_56.119221/model-00

### Model 2e : Resize to 120*120, agumentation, flipped images, normalisation, cropping, No edge detection

In [18]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d2(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________

34/34 [==============================] - 6s 191ms/step - loss: 1.6059 - categorical_accuracy: 0.2222 - val_loss: 1.6192 - val_categorical_accuracy: 0.2100

Epoch 00017: saving model to model_init_2018-10-2121_42_40.379231/model-00017-1.60586-0.22222-1.61922-0.21000.h5
Epoch 18/20
34/34 [==============================] - 6s 177ms/step - loss: 1.6099 - categorical_accuracy: 0.2092 - val_loss: 1.6192 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2121_42_40.379231/model-00018-1.60995-0.20915-1.61924-0.21000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 19/20
34/34 [==============================] - 6s 187ms/step - loss: 1.6143 - categorical_accuracy: 0.1471 - val_loss: 1.6192 - val_categorical_accuracy: 0.2100

Epoch 00019: saving model to model_init_2018-10-2121_42_40.379231/model-00019-1.61430-0.14706-1.61924-0.21000.h5
Epoch 20/20
34/34 [==============================] - 6s 191ms/step - loss: 1.6115 - cate

### Model 2f : Resize to 120*120, agumentation, flipped images, normalisation, cropping, edge detection

In [19]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d2(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
______________________________________________________

34/34 [==============================] - 7s 200ms/step - loss: 1.6078 - categorical_accuracy: 0.2190 - val_loss: 1.8643 - val_categorical_accuracy: 0.2200

Epoch 00017: saving model to model_init_2018-10-2121_45_24.174074/model-00017-1.60776-0.21895-1.86433-0.22000.h5
Epoch 18/20
34/34 [==============================] - 7s 192ms/step - loss: 1.6049 - categorical_accuracy: 0.2222 - val_loss: 1.8645 - val_categorical_accuracy: 0.2200

Epoch 00018: saving model to model_init_2018-10-2121_45_24.174074/model-00018-1.60493-0.22222-1.86451-0.22000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 19/20
34/34 [==============================] - 7s 216ms/step - loss: 1.6135 - categorical_accuracy: 0.1797 - val_loss: 1.8643 - val_categorical_accuracy: 0.2200

Epoch 00019: saving model to model_init_2018-10-2121_45_24.174074/model-00019-1.61353-0.17974-1.86435-0.22000.h5
Epoch 20/20
34/34 [==============================] - 7s 220ms/step - loss: 1.6081 - cate

## Model #3

### Model 3a : Resize to 120*120,  No agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [20]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
______________________________________________________

34/34 [==============================] - 5s 141ms/step - loss: 1.6328 - categorical_accuracy: 0.3235 - val_loss: 1.3974 - val_categorical_accuracy: 0.4300

Epoch 00017: saving model to model_init_2018-10-2121_48_25.184784/model-00017-1.63283-0.32353-1.39743-0.43000.h5
Epoch 18/20
34/34 [==============================] - 5s 145ms/step - loss: 1.6698 - categorical_accuracy: 0.2745 - val_loss: 1.4373 - val_categorical_accuracy: 0.4100

Epoch 00018: saving model to model_init_2018-10-2121_48_25.184784/model-00018-1.66982-0.27451-1.43726-0.41000.h5
Epoch 19/20
34/34 [==============================] - 5s 148ms/step - loss: 1.6766 - categorical_accuracy: 0.3725 - val_loss: 1.4797 - val_categorical_accuracy: 0.4200

Epoch 00019: saving model to model_init_2018-10-2121_48_25.184784/model-00019-1.67660-0.37255-1.47974-0.42000.h5

Epoch 00019: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 20/20
34/34 [==============================] - 5s 141ms/step - loss: 1.7473 - categ

### Model 3b : Resize to 120*120,   agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [21]:
train_gen = DataGenerator(affine=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
______________________________________________________

34/34 [==============================] - 6s 173ms/step - loss: 1.5890 - categorical_accuracy: 0.3775 - val_loss: 1.5777 - val_categorical_accuracy: 0.3600

Epoch 00016: saving model to model_init_2018-10-2121_50_16.316011/model-00016-1.58900-0.37745-1.57772-0.36000.h5
Epoch 17/20
34/34 [==============================] - 6s 170ms/step - loss: 1.4130 - categorical_accuracy: 0.4608 - val_loss: 1.4423 - val_categorical_accuracy: 0.3800

Epoch 00017: saving model to model_init_2018-10-2121_50_16.316011/model-00017-1.41300-0.46078-1.44233-0.38000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 18/20
34/34 [==============================] - 5s 159ms/step - loss: 1.6017 - categorical_accuracy: 0.3137 - val_loss: 1.1397 - val_categorical_accuracy: 0.5000

Epoch 00018: saving model to model_init_2018-10-2121_50_16.316011/model-00018-1.60174-0.31373-1.13965-0.50000.h5
Epoch 19/20
34/34 [==============================] - 5s 160ms/step - loss: 1.5842 - cate

### Model 3c : Resize to 120*120,  agumentation, flipped images, No normalisation, No cropping, No edge detection

In [22]:
train_gen = DataGenerator(affine=True, flip=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

batch_size = 10
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 10
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
______________________________________________________


Epoch 00016: saving model to model_init_2018-10-2121_52_37.183439/model-00016-0.85755-0.66667-0.66565-0.75000.h5
Epoch 17/20
67/67 [==============================] - 14s 207ms/step - loss: 0.9028 - categorical_accuracy: 0.6186 - val_loss: 0.6981 - val_categorical_accuracy: 0.7500

Epoch 00017: saving model to model_init_2018-10-2121_52_37.183439/model-00017-0.90275-0.61857-0.69809-0.75000.h5
Epoch 18/20
67/67 [==============================] - 14s 204ms/step - loss: 0.9631 - categorical_accuracy: 0.6070 - val_loss: 0.7379 - val_categorical_accuracy: 0.7300

Epoch 00018: saving model to model_init_2018-10-2121_52_37.183439/model-00018-0.96313-0.60697-0.73785-0.73000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 19/20
67/67 [==============================] - 15s 217ms/step - loss: 0.7894 - categorical_accuracy: 0.6700 - val_loss: 0.6748 - val_categorical_accuracy: 0.7500

Epoch 00019: saving model to model_init_2018-10-2121_52_37.183439/model-0

### Model 3d : Resize to 120*120,  agumentation, flipped images, normalisation, No cropping, No edge detection

In [23]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 10
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
______________________________________________________

67/67 [==============================] - 15s 218ms/step - loss: 0.5150 - categorical_accuracy: 0.7761 - val_loss: 9.3195 - val_categorical_accuracy: 0.4200

Epoch 00016: saving model to model_init_2018-10-2121_57_21.165248/model-00016-0.51501-0.77612-9.31948-0.42000.h5
Epoch 17/20
67/67 [==============================] - 13s 201ms/step - loss: 0.5455 - categorical_accuracy: 0.7761 - val_loss: 9.1724 - val_categorical_accuracy: 0.4300

Epoch 00017: saving model to model_init_2018-10-2121_57_21.165248/model-00017-0.54545-0.77612-9.17243-0.43000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 18/20
67/67 [==============================] - 14s 216ms/step - loss: 0.5745 - categorical_accuracy: 0.7745 - val_loss: 9.3027 - val_categorical_accuracy: 0.4200

Epoch 00018: saving model to model_init_2018-10-2121_57_21.165248/model-00018-0.57448-0.77446-9.30272-0.42000.h5
Epoch 19/20
67/67 [==============================] - 13s 198ms/step - loss: 0.5057 -

### Model 3e : Resize to 120*120,  agumentation, flipped images, normalisation,  cropping, No edge detection

In [24]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 10
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
______________________________________________________


Epoch 00015: saving model to model_init_2018-10-2122_02_27.244563/model-00015-1.42030-0.41791-12.24975-0.24000.h5

Epoch 00015: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 16/20
67/67 [==============================] - 13s 194ms/step - loss: 1.3845 - categorical_accuracy: 0.4395 - val_loss: 12.2498 - val_categorical_accuracy: 0.2400

Epoch 00016: saving model to model_init_2018-10-2122_02_27.244563/model-00016-1.38453-0.43947-12.24975-0.24000.h5
Epoch 17/20
67/67 [==============================] - 13s 189ms/step - loss: 1.3109 - categorical_accuracy: 0.4527 - val_loss: 12.2498 - val_categorical_accuracy: 0.2400

Epoch 00017: saving model to model_init_2018-10-2122_02_27.244563/model-00017-1.31091-0.45274-12.24975-0.24000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 18/20
67/67 [==============================] - 14s 216ms/step - loss: 1.3556 - categorical_accuracy: 0.4262 - val_loss: 12.2498 - val_categorical_acc

### Model 3f : Resize to 120*120,  agumentation, flipped images, normalisation, cropping, edge detection

In [25]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 10
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 40, 40, 16)    0         
______________________________________________________


Epoch 00015: saving model to model_init_2018-10-2122_07_26.449958/model-00015-0.45636-0.79602-10.88795-0.31000.h5
Epoch 16/20
67/67 [==============================] - 15s 223ms/step - loss: 0.5416 - categorical_accuracy: 0.7728 - val_loss: 10.7817 - val_categorical_accuracy: 0.3100

Epoch 00016: saving model to model_init_2018-10-2122_07_26.449958/model-00016-0.54161-0.77280-10.78175-0.31000.h5

Epoch 00016: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 17/20
67/67 [==============================] - 15s 229ms/step - loss: 0.5810 - categorical_accuracy: 0.7595 - val_loss: 10.9265 - val_categorical_accuracy: 0.3100

Epoch 00017: saving model to model_init_2018-10-2122_07_26.449958/model-00017-0.58102-0.75954-10.92646-0.31000.h5
Epoch 18/20
67/67 [==============================] - 15s 222ms/step - loss: 0.5594 - categorical_accuracy: 0.7479 - val_loss: 10.9424 - val_categorical_accuracy: 0.2900

Epoch 00018: saving model to model_init_2018-10-2122_07_26.449958/

## Model #4

### Model 4a : Resize to 120*120,  No agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [44]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

batch_size = 90
num_epochs = 10

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 90
# epochs = 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
______________________________________________________

### Model 4b : Resize to 120*120,  agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [48]:
train_gen = DataGenerator(affine=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

batch_size = 50
num_epochs = 10

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 50
# epochs = 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 16)    64        
______________________________________________________

### Model 4c : Resize to 120*120,  agumentation,  flipped images, No normalisation, No cropping, No edge detection

In [52]:
train_gen = DataGenerator(affine=True, flip=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

batch_size = 30
num_epochs = 10

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 30
# epochs = 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_10 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_11 (Batc (None, 15, 60, 60, 16)    64        
______________________________________________________

### Model 4d : Resize to 120*120,  agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [57]:
train_gen = DataGenerator(affine=True, crop=False)
val_gen = DataGenerator(crop=False)
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

batch_size = 40
num_epochs = 10

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 40
# epochs = 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_17 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 60, 60, 16)    64        
______________________________________________________

### Model 4e : Resize to 120*120,  agumentation, flipped images, normalisation, cropping, No edge detection

In [30]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
______________________________________________________

34/34 [==============================] - 7s 195ms/step - loss: 0.8749 - categorical_accuracy: 0.6601 - val_loss: 10.3160 - val_categorical_accuracy: 0.3600

Epoch 00017: saving model to model_init_2018-10-2122_23_29.778726/model-00017-0.87489-0.66013-10.31597-0.36000.h5
Epoch 18/20
34/34 [==============================] - 7s 198ms/step - loss: 0.9467 - categorical_accuracy: 0.6111 - val_loss: 10.4522 - val_categorical_accuracy: 0.3500

Epoch 00018: saving model to model_init_2018-10-2122_23_29.778726/model-00018-0.94675-0.61111-10.45221-0.35000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 19/20
34/34 [==============================] - 7s 198ms/step - loss: 0.8816 - categorical_accuracy: 0.6765 - val_loss: 10.4901 - val_categorical_accuracy: 0.3400

Epoch 00019: saving model to model_init_2018-10-2122_23_29.778726/model-00019-0.88163-0.67647-10.49010-0.34000.h5
Epoch 20/20
34/34 [==============================] - 7s 205ms/step - loss: 0.8582

### Model 4f : Resize to 120*120,  agumentation, flipped images, normalisation, cropping, edge detection

In [31]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
______________________________________________________

34/34 [==============================] - 8s 243ms/step - loss: 1.2987 - categorical_accuracy: 0.4869 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00016: saving model to model_init_2018-10-2122_26_23.431341/model-00016-1.29874-0.48693-12.41093-0.23000.h5
Epoch 17/20
34/34 [==============================] - 8s 230ms/step - loss: 1.3019 - categorical_accuracy: 0.4575 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00017: saving model to model_init_2018-10-2122_26_23.431341/model-00017-1.30189-0.45752-12.41093-0.23000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 18/20
34/34 [==============================] - 7s 214ms/step - loss: 1.3219 - categorical_accuracy: 0.4771 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00018: saving model to model_init_2018-10-2122_26_23.431341/model-00018-1.32187-0.47712-12.41093-0.23000.h5
Epoch 19/20
34/34 [==============================] - 8s 226ms/step - loss: 1.3829 

In [40]:
### Model 4g : Resize to 120*120,  agumentation, flipped images, No normalisation, cropping, edge detection

In [41]:
train_gen = DataGenerator(affine=True, flip=True, normalize=False, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
______________________________________________________

34/34 [==============================] - 8s 242ms/step - loss: 1.4066 - categorical_accuracy: 0.3954 - val_loss: 7.0784 - val_categorical_accuracy: 0.2300

Epoch 00016: saving model to model_init_2018-10-2122_49_34.968496/model-00016-1.40665-0.39542-7.07836-0.23000.h5

Epoch 00016: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 17/20
34/34 [==============================] - 9s 258ms/step - loss: 1.4472 - categorical_accuracy: 0.3791 - val_loss: 7.1558 - val_categorical_accuracy: 0.2300

Epoch 00017: saving model to model_init_2018-10-2122_49_34.968496/model-00017-1.44721-0.37908-7.15582-0.23000.h5
Epoch 18/20
34/34 [==============================] - 9s 253ms/step - loss: 1.4193 - categorical_accuracy: 0.3856 - val_loss: 7.1740 - val_categorical_accuracy: 0.2300

Epoch 00018: saving model to model_init_2018-10-2122_49_34.968496/model-00018-1.41928-0.38562-7.17398-0.23000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 1

## Model #5

### Model 5a : Resize to 120*120,  No agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [32]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 5s 145ms/step - loss: 13.1157 - categorical_accuracy: 0.1863 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00016: saving model to model_init_2018-10-2122_29_41.675097/model-00016-13.11570-0.18627-12.41093-0.23000.h5

Epoch 00016: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 17/20
34/34 [==============================] - 5s 142ms/step - loss: 13.5898 - categorical_accuracy: 0.1569 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00017: saving model to model_init_2018-10-2122_29_41.675097/model-00017-13.58977-0.15686-12.41093-0.23000.h5
Epoch 18/20
34/34 [==============================] - 5s 143ms/step - loss: 12.4836 - categorical_accuracy: 0.2255 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00018: saving model to model_init_2018-10-2122_29_41.675097/model-00018-12.48362-0.22549-12.41093-0.23000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e

### Model 5b : Resize to 120*120,  agumentation, No flipped images, No normalisation, No cropping, No edge detection

In [33]:
train_gen = DataGenerator(affine=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 6s 180ms/step - loss: 12.4996 - categorical_accuracy: 0.2206 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00016: saving model to model_init_2018-10-2122_31_35.281471/model-00016-12.49962-0.22059-12.73330-0.21000.h5

Epoch 00016: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 17/20
34/34 [==============================] - 6s 177ms/step - loss: 12.2466 - categorical_accuracy: 0.2402 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00017: saving model to model_init_2018-10-2122_31_35.281471/model-00017-12.24659-0.24020-12.73330-0.21000.h5
Epoch 18/20
34/34 [==============================] - 6s 171ms/step - loss: 12.6416 - categorical_accuracy: 0.2157 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2122_31_35.281471/model-00018-12.64164-0.21569-12.73330-0.21000.h5

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e

### Model 5c : Resize to 120*120,  agumentation, flipped images, No normalisation, No cropping, No edge detection

In [34]:
train_gen = DataGenerator(affine=True, flip=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 7s 216ms/step - loss: 12.8523 - categorical_accuracy: 0.2026 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00016: saving model to model_init_2018-10-2122_34_00.297517/model-00016-12.85234-0.20261-12.73330-0.21000.h5
Epoch 17/20
34/34 [==============================] - 7s 192ms/step - loss: 13.2737 - categorical_accuracy: 0.1765 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00017: saving model to model_init_2018-10-2122_34_00.297517/model-00017-13.27373-0.17647-12.73330-0.21000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 18/20
34/34 [==============================] - 7s 208ms/step - loss: 12.6945 - categorical_accuracy: 0.2124 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2122_34_00.297517/model-00018-12.69450-0.21242-12.73330-0.21000.h5
Epoch 19/20
34/34 [==============================] - 7s 220ms/step - loss: 1

### Model 5d : Resize to 120*120,  agumentation, flipped images, normalisation, No cropping, No edge detection

In [35]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 7s 206ms/step - loss: 1.1996 - categorical_accuracy: 0.4477 - val_loss: 9.4374 - val_categorical_accuracy: 0.3900

Epoch 00016: saving model to model_init_2018-10-2122_36_55.673460/model-00016-1.19958-0.44771-9.43739-0.39000.h5
Epoch 17/20
34/34 [==============================] - 6s 190ms/step - loss: 1.1588 - categorical_accuracy: 0.4967 - val_loss: 9.4813 - val_categorical_accuracy: 0.3900

Epoch 00017: saving model to model_init_2018-10-2122_36_55.673460/model-00017-1.15883-0.49673-9.48133-0.39000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 18/20
34/34 [==============================] - 7s 200ms/step - loss: 1.2891 - categorical_accuracy: 0.4314 - val_loss: 9.4048 - val_categorical_accuracy: 0.4000

Epoch 00018: saving model to model_init_2018-10-2122_36_55.673460/model-00018-1.28909-0.43137-9.40477-0.40000.h5
Epoch 19/20
34/34 [==============================] - 7s 196ms/step - loss: 1.0779 - cate

### Model 5e : Resize to 120*120,  agumentation, flipped images, normalisation, cropping, No edge detection

In [36]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 7s 200ms/step - loss: 1.2721 - categorical_accuracy: 0.4575 - val_loss: 10.3227 - val_categorical_accuracy: 0.3600

Epoch 00016: saving model to model_init_2018-10-2122_39_49.545996/model-00016-1.27207-0.45752-10.32265-0.36000.h5
Epoch 17/20
34/34 [==============================] - 7s 200ms/step - loss: 1.1835 - categorical_accuracy: 0.5229 - val_loss: 10.3245 - val_categorical_accuracy: 0.3600

Epoch 00017: saving model to model_init_2018-10-2122_39_49.545996/model-00017-1.18347-0.52288-10.32447-0.36000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 18/20
34/34 [==============================] - 7s 206ms/step - loss: 1.1870 - categorical_accuracy: 0.5392 - val_loss: 10.3232 - val_categorical_accuracy: 0.3600

Epoch 00018: saving model to model_init_2018-10-2122_39_49.545996/model-00018-1.18705-0.53922-10.32318-0.36000.h5
Epoch 19/20
34/34 [==============================] - 7s 191ms/step - loss: 1.2425 

### Model 5f : Resize to 120*120, All

In [37]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 8s 222ms/step - loss: 1.3339 - categorical_accuracy: 0.4085 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00016: saving model to model_init_2018-10-2122_42_43.394553/model-00016-1.33390-0.40850-12.41093-0.23000.h5
Epoch 17/20
34/34 [==============================] - 8s 239ms/step - loss: 1.3429 - categorical_accuracy: 0.3791 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00017: saving model to model_init_2018-10-2122_42_43.394553/model-00017-1.34289-0.37908-12.41093-0.23000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 18/20
34/34 [==============================] - 7s 220ms/step - loss: 1.3523 - categorical_accuracy: 0.4085 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00018: saving model to model_init_2018-10-2122_42_43.394553/model-00018-1.35233-0.40850-12.41093-0.23000.h5
Epoch 19/20
34/34 [==============================] - 8s 226ms/step - loss: 1.3093 

### Model 5g : Resize to 120*120, All except normalization

In [39]:
train_gen = DataGenerator(affine=True, flip=True, normalize=False, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
______________________________________________________

34/34 [==============================] - 9s 257ms/step - loss: 13.0104 - categorical_accuracy: 0.1928 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00016: saving model to model_init_2018-10-2122_46_00.151324/model-00016-13.01036-0.19281-12.73330-0.21000.h5
Epoch 17/20
34/34 [==============================] - 8s 232ms/step - loss: 12.0330 - categorical_accuracy: 0.2516 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00017: saving model to model_init_2018-10-2122_46_00.151324/model-00017-12.03296-0.25163-12.73330-0.21000.h5

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 18/20
34/34 [==============================] - 8s 242ms/step - loss: 13.0630 - categorical_accuracy: 0.1895 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00018: saving model to model_init_2018-10-2122_46_00.151324/model-00018-13.06303-0.18954-12.73329-0.21000.h5
Epoch 19/20
34/34 [==============================] - 8s 241ms/step - loss: 1

### Model 5h : Resize to 120*120, All except normalization

In [ ]:
train_gen = DataGenerator(affine=True, flip=True, normalize=False, crop=True, edge=False)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)

#batch_size = 20
#num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)